# Yahoo Finance Stock Screener Using Daily Data & Indicators

## Uses the following indicators
* SMA: https://www.investopedia.com/terms/s/sma.asp
* RSI: https://www.investopedia.com/terms/r/rsi.asp
* Bollinger Bands: https://www.investopedia.com/articles/technical/102201.asp & https://www.investopedia.com/trading/using-bollinger-bands-to-gauge-trends/

## Overview 
* Process will pull OHLC data from yahoo finance for a few tickers
* Based on the idicators Buy, Sell, Neutral signals will be generated for the day

### Get OHLC data from yahoo finance and turn into dataframe

In [1]:
# import libraries
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

In [2]:
# Define the symbols and date range
symbols = ['MMM', 'AAPL', 'GOOGL']  # Example list of symbols
start_date = '2022-12-01'

# Get the current date
current_date = datetime.today()
end_date = (current_date - timedelta(days=1)).strftime('%Y-%m-%d')

# Create an empty list to store the results
results = []

# Loop over each symbol
for symbol in symbols:
    # Fetch OHLC data from Yahoo Finance
    data = yf.download(symbol, start=start_date, end=end_date)

    # Calculate SMA
    period = 20
    data['SMA'] = data['Close'].rolling(window=period, min_periods=20).mean().shift(1)

    # Calculate RSI
    period = 14
    delta = data['Close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=period).mean().shift(1)
    avg_loss = loss.rolling(window=period).mean().shift(1)
    relative_strength = avg_gain / avg_loss
    data['RSI'] = 100 - (100 / (1 + relative_strength))

    # Calculate Bollinger Bands
    period = 20
    std_dev = data['Close'].rolling(window=period).std().shift(1)
    data['Upper Band'] = data['SMA'] + 2 * std_dev
    data['Middle Band'] = data['SMA']
    data['Lower Band'] = data['SMA'] - 2 * std_dev

    # Calculate Signal columns
    data['SMA Signal'] = ''
    data['RSI Signal'] = ''
    data['BB Signal'] = ''

    data.loc[data['SMA'] > data['Close'], 'SMA Signal'] = 'Sell'
    data.loc[data['SMA'] < data['Close'], 'SMA Signal'] = 'Buy'
    data.loc[data['SMA'] == data['Close'], 'SMA Signal'] = 'Neutral'

    data.loc[data['RSI'] > 70, 'RSI Signal'] = 'Sell'
    data.loc[data['RSI'] < 30, 'RSI Signal'] = 'Buy'
    data.loc[(data['RSI'] >= 30) & (data['RSI'] <= 70), 'RSI Signal'] = 'Neutral'

    data.loc[(data['Close'] < data['Lower Band']) & (data['RSI'] < 30), 'BB Signal'] = 'Buy'
    data.loc[(data['Close'] > data['Upper Band']) & (data['RSI'] > 70), 'BB Signal'] = 'Sell'
    data.loc[((data['Close'] >= data['Lower Band']) & (data['Close'] <= data['Middle Band'])) | ((data['Close'] <= data['Upper Band']) & (data['Close'] >= data['Middle Band'])), 'BB Signal'] = 'Neutral'

    # Calculate Buy, Sell, and Neutral Signal counts
    data['Buy Signal'] = (data['SMA Signal'] == 'Buy').astype(int) + (data['RSI Signal'] == 'Buy').astype(int) + (data['BB Signal'] == 'Buy').astype(int)
    data['Sell Signal'] = (data['SMA Signal'] == 'Sell').astype(int) + (data['RSI Signal'] == 'Sell').astype(int) + (data['BB Signal'] == 'Sell').astype(int)
    data['Neutral Signal'] = (data['SMA Signal'] == 'Neutral').astype(int) + (data['RSI Signal'] == 'Neutral').astype(int) + (data['BB Signal'] == 'Neutral').astype(int)

    # Add the symbol and its data to the results list
    results.append({'Symbol': symbol, 'Data': data})

# Convert results to dataframes
dataframes = []
for result in results:
    symbol = result['Symbol']
    data = result['Data']
    dataframe = pd.DataFrame(data)
    dataframe['Symbol'] = symbol
    dataframes.append(dataframe)

# Combine all dataframes into a single dataframe
combined_dataframe = pd.concat(dataframes)
# combined_dataframe.head(35)

# Sort by Symbol and then Date descending
combined_dataframe.sort_values(by=['Symbol', 'Date'], ascending=[True, False], inplace=True)

# Display the combined dataframe
combined_dataframe.head()


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,SMA,RSI,Upper Band,Middle Band,Lower Band,SMA Signal,RSI Signal,BB Signal,Buy Signal,Sell Signal,Neutral Signal,Symbol
Date,,,,,,,,,,,,,,,,,,
2023-07-17,191.899994,194.320007,191.809998,193.990005,193.990005,50520200,188.684499,60.245310,194.287993,188.684499,183.081005,Buy,Neutral,Neutral,1,0,2,AAPL
2023-07-14,190.229996,191.179993,189.630005,190.690002,190.690002,41573900,188.347499,58.966560,194.246081,188.347499,182.448917,Buy,Neutral,Neutral,1,0,2,AAPL
2023-07-13,190.500000,191.190002,189.779999,190.539993,190.539993,41342300,187.985999,63.198553,194.196750,187.985999,181.775249,Buy,Neutral,Neutral,1,0,2,AAPL
2023-07-12,189.679993,191.699997,188.470001,189.770004,189.770004,60750200,187.686999,57.182996,194.108337,187.686999,181.265660,Buy,Neutral,Neutral,1,0,2,AAPL
2023-07-11,189.160004,189.300003,186.600006,188.080002,188.080002,46638100,187.330999,58.815120,194.415791,187.330999,180.246207,Buy,Neutral,Neutral,1,0,2,AAPL


### Take the above data frame and show the 5 most recent days per symbol

In [3]:
# Sort the combined_dataframe by 'Symbol' and 'Date'
sorted_dataframe = combined_dataframe.sort_values(by=['Symbol', 'Date'], ascending=[True, False])

# Extract the most recent 5 days of data per ticker
grouped_dataframe = sorted_dataframe.groupby('Symbol').head(5)

print("Most recent 5 days of data per ticker:")
grouped_dataframe


Most recent 5 days of data per ticker:


,Open,High,Low,Close,Adj Close,Volume,SMA,RSI,Upper Band,Middle Band,Lower Band,SMA Signal,RSI Signal,BB Signal,Buy Signal,Sell Signal,Neutral Signal,Symbol
Date,,,,,,,,,,,,,,,,,,
2023-07-17,191.899994,194.320007,191.809998,193.990005,193.990005,50520200,188.684499,60.245310,194.287993,188.684499,183.081005,Buy,Neutral,Neutral,1,0,2,AAPL
2023-07-14,190.229996,191.179993,189.630005,190.690002,190.690002,41573900,188.347499,58.966560,194.246081,188.347499,182.448917,Buy,Neutral,Neutral,1,0,2,AAPL
2023-07-13,190.500000,191.190002,189.779999,190.539993,190.539993,41342300,187.985999,63.198553,194.196750,187.985999,181.775249,Buy,Neutral,Neutral,1,0,2,AAPL
2023-07-12,189.679993,191.699997,188.470001,189.770004,189.770004,60750200,187.686999,57.182996,194.108337,187.686999,181.265660,Buy,Neutral,Neutral,1,0,2,AAPL
2023-07-11,189.160004,189.300003,186.600006,188.080002,188.080002,46638100,187.330999,58.815120,194.415791,187.330999,180.246207,Buy,Neutral,Neutral,1,0,2,AAPL
2023-07-17,125.940002,127.099998,124.199997,124.650002,124.650002,25716200,120.856499,56.454321,126.103145,120.856499,115.609854,Buy,Neutral,Neutral,1,0,2,GOOGL
2023-07-14,124.800003,126.779999,123.489998,125.419998,125.419998,33266800,120.768999,52.921393,125.746671,120.768999,115.791328,Buy,Neutral,Neutral,1,0,2,GOOGL
2023-07-13,120.930000,124.830002,120.449997,124.540001,124.540001,44297900,120.733500,46.102016,125.606969,120.733500,115.860030,Buy,Neutral,Neutral,1,0,2,GOOGL
2023-07-12,118.800003,120.330002,118.410004,118.930000,118.930000,30404400,120.968999,36.165277,125.929937,120.968999,116.008062,Sell,Neutral,Neutral,0,1,2,GOOGL
